# Part 1


In [ ]:
import itertools


def find_num_alternatives(row, lengths):
    num_alternatives = 0
    qmark_positions = [i for i, char in enumerate(row) if char == "?"]
    num_missing = sum(lengths) - row.count("#")
    for qmark_replace in itertools.combinations(qmark_positions, num_missing):
        new_row = list(row)
        for pos in qmark_replace:
            new_row[pos] = "#"
        alternative = "".join(new_row).replace("?", ".")
        new_lenghts = [len(chars) for chars in alternative.split(".") if chars]
        if new_lenghts == lengths:
            num_alternatives += 1
    return num_alternatives

In [ ]:
with open("day12_input.txt") as file:
    alternatives = []
    for i, line in enumerate(file):
        row, digits = line.strip().split(" ")
        lengths = [int(d) for d in digits.split(",")]
        alternatives.append(find_num_alternatives(row, lengths))

print("Answer:", sum(alternatives))

# Part 2


In [ ]:
from functools import cache


@cache
def dp_num_alternatives(chars: str, groups: tuple[int, ...]) -> int:
    # Handle base cases, at the end of the recursion:
    if not groups:
        # We have consumed all groups: If there are no `#`s left, this is a valid
        # solution.
        if "#" not in chars:
            return 1
        # Otherwise, this is not a valid solution
        return 0

    if not chars:
        # We have consumed all chars, but we still have groups left: This is not a valid
        # solution
        return 0

    def consume_this_group() -> int:
        # We are at the start of a group.
        # 1. The number of available characters must be at least `group_length`
        # 2. There can be no `.` in the first `group_length` characters.
        # 3. The next character after the group cannot be a `#`:
        group_length = groups[0]
        if (
            (len(chars) < group_length)
            or ("." in chars[:group_length])
            or (chars[group_length : group_length + 1] == "#")
        ):
            return 0
        # Otherwise, consume the group and continue
        return dp_num_alternatives(chars[group_length + 1 :], groups[1:])

    # Depending on the first character, we have different options:
    match chars[0]:
        case "#":
            # Try to consume the group
            return consume_this_group()

        case ".":
            # Skip this character, and keep the groups
            return dp_num_alternatives(chars[1:], groups)

        case "?":
            # We can either treat this as a `.` and skip it, or handle it as a `#`:
            return dp_num_alternatives(chars[1:], groups) + consume_this_group()

In [ ]:
with open("day12_input.txt") as file:
    alternatives = []
    for i, line in enumerate(file):
        row, digits = line.strip().split(" ")
        row = "?".join([row] * 5)
        lengths = tuple(int(d) for d in digits.split(",")) * 5
        alternatives.append(dp_num_alternatives(row, lengths))

print("Answer:", sum(alternatives))